In [1]:
import numpy as np
from ldpc.codes import ring_code
from ldpc.code_util import compute_code_distance
import ldpc.mod2 as mod2
from ldpc import protograph as pt
from ldpc.codes import hamming_code
from bposd.css import css_code
from bposd.hgp import hgp
from css_ss_decode_sim import css_ss_decode_sim
from lifted_hgp import *
#from bposd.css_decode_sim import css_decode_sim

In [2]:
H=pt.array([[(0,1)]])
qcode=lifted_hgp(lift_parameter=4,a=H, b=H)
qcode.test()

[[1 0 0 1]
 [1 1 0 0]
 [0 1 1 0]
 [0 0 1 1]
 [1 1 0 0]
 [0 1 1 0]
 [0 0 1 1]
 [1 0 0 1]]
[[1 1 0 0 1 0 0 1]
 [0 1 1 0 1 1 0 0]
 [0 0 1 1 0 1 1 0]
 [1 0 0 1 0 0 1 1]]
delta_0_2d.shape: (4, 8)
delta_m1_2d.shape: (8, 4)
<Unnamed CSS code>
 -Block dimensions: Pass
 -PCMs commute hz@hx.T==0: Pass
 -PCMs commute hx@hz.T==0: Pass
 -lx \in ker{hz} AND lz \in ker{hx}: Pass
 -lx and lz anticommute: Pass
 -<Unnamed CSS code> is a valid CSS code w/ params [8,2,nan]


True

In [3]:
sim_input={
    "error_rate": 0.10, #the physical error rate on the qubits
    "target_runs": 1000, #the number of cycles to simulate
    "bp_method": "minimum_sum", #the bp method
    "ms_scaling_factor": 0.625, # the min-sum scaling factor
    "osd_method": "osd_e", # OSD method
    "osd_order": 10, #OSD order
    "xyz_error_bias": [1,1,1], #the relative XYZ bias
    
    "sector_list": [0, 1, 2],
    "rotated_sectors": [],
    "sector_lengths": qcode.sector_lengths,

    "p_meas_err": 0.1,
    
    "channel_update": "x->z", # the channel update orientation
    'max_iter': int(qcode.N/10), #the interation depth for BP
    'tqdm_disable': False #show live stas
}

In [4]:
qcode.mz_4d.shape

(64, 16)

In [5]:
css_ss_decode_sim(hx = qcode.hx_4d, hz = qcode.hz_4d.T, 
                  mx = qcode.mx_4d, mz = qcode.mz_4d.T, 
                  **sim_input)

RNG Seed: 1398605097
<class 'numpy.ndarray'>
Constructing CSS code from hx and hz matrices...
Checking the CSS code is valid...
<Unnamed CSS code>
 -Block dimensions: Pass
 -PCMs commute hz@hx.T==0: Pass
 -PCMs commute hx@hz.T==0: Pass
 -lx \in ker{hz} AND lz \in ker{hx}: Pass
 -lx and lz anticommute: Pass
 -<Unnamed CSS code> is a valid CSS code w/ params [96,6,nan]


d_max: 1; OSDW_WER: 52.8±0.055%; OSDW: 98.9±0.33%; OSD0: 99.4±0.24%;: 100% 1000/1000 [00:08<00:00, 120.85it/s]
